<a href="https://colab.research.google.com/github/subhobrata/deep-learning-for-visual-computing/blob/master/Lecture_17a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import torch
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.autograd import Function
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
transform = transforms.Compose([transforms.ToTensor()])
BatchSize = 1000

trainset = torchvision.datasets.MNIST(root='./MNIST', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BatchSize,
                                          shuffle=True, num_workers=4) # Creating dataloader

testset = torchvision.datasets.MNIST(root='./MNIST', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BatchSize,
                                         shuffle=False, num_workers=4) # Creating dataloader

classes = ('zero', 'one', 'two', 'three',
           'four', 'five', 'six', 'seven', 'eight', 'nine')

Processing...
Done!


In [3]:
# Check availability of GPU
use_gpu = torch.cuda.is_available()
if use_gpu:
    print('GPU is available!')

GPU is available!


In [4]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 400),
            nn.Tanh())
        self.decoder = nn.Sequential(
            nn.Linear(400, 28*28),
            nn.Sigmoid())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


net = autoencoder()
print(net)

if use_gpu:
    net = net.double().cuda()
else:
    net = net.double()

autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=400, bias=True)
    (1): Tanh()
  )
  (decoder): Sequential(
    (0): Linear(in_features=400, out_features=784, bias=True)
    (1): Sigmoid()
  )
)


In [5]:
iterations = 10
learning_rate = 0.98
criterion = nn.MSELoss()


for epoch in range(iterations):  # loop over the dataset multiple times
    runningLoss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        # wrap them in Variable
        if use_gpu:
            inputs = Variable(inputs.view(-1, 28*28).double()).cuda()
        else:
            inputs = Variable(inputs.view(-1, 28*28).double())

        net.zero_grad()  # zeroes the gradient buffers of all parameters
        outputs = net(inputs) # forward 
        loss = criterion(outputs, inputs) # calculate loss
        loss.backward() #  backpropagate the loss
        for f in net.parameters():
            f.data.sub_(f.grad.data * learning_rate) # weight = weight - learning_rate * gradient (Update Weights)
        runningLoss += loss.item()
        
    print('At Iteration : %d / %d  ;  Mean-Squared Error : %f'%(epoch + 1,iterations,
                                                                        runningLoss/(60000/BatchSize)))
print('Finished Training')

At Iteration : 1 / 10  ;  Mean-Squared Error : 0.191208
At Iteration : 2 / 10  ;  Mean-Squared Error : 0.100919
At Iteration : 3 / 10  ;  Mean-Squared Error : 0.077720
At Iteration : 4 / 10  ;  Mean-Squared Error : 0.072498
At Iteration : 5 / 10  ;  Mean-Squared Error : 0.070350
At Iteration : 6 / 10  ;  Mean-Squared Error : 0.069102
At Iteration : 7 / 10  ;  Mean-Squared Error : 0.068209
At Iteration : 8 / 10  ;  Mean-Squared Error : 0.067472
At Iteration : 9 / 10  ;  Mean-Squared Error : 0.066801
At Iteration : 10 / 10  ;  Mean-Squared Error : 0.066149
Finished Training


In [6]:
# Adding New Layer (Stacking)
net.encoder.add_module('New_Encoder_Layer', nn.Sequential(nn.Linear(400, 256),nn.Tanh()))
net.encoder.add_module('New_Decoder_Layer', nn.Sequential(nn.Linear(256, 400),nn.Tanh()))
print(net)
if use_gpu:
    net = net.double().cuda()
else:
    net = net.double()

autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=400, bias=True)
    (1): Tanh()
    (New_Encoder_Layer): Sequential(
      (0): Linear(in_features=400, out_features=256, bias=True)
      (1): Tanh()
    )
    (New_Decoder_Layer): Sequential(
      (0): Linear(in_features=256, out_features=400, bias=True)
      (1): Tanh()
    )
  )
  (decoder): Sequential(
    (0): Linear(in_features=400, out_features=784, bias=True)
    (1): Sigmoid()
  )
)


In [7]:
for epoch in range(iterations):  # loop over the dataset multiple times
    runningLoss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        # wrap them in Variable
        if use_gpu:
            inputs = Variable(inputs.view(-1, 28*28).double()).cuda()
        else:
            inputs = Variable(inputs.view(-1, 28*28).double())

        net.zero_grad()  # zeroes the gradient buffers of all parameters
        outputs = net(inputs) # forward 
        loss = criterion(outputs, inputs) # calculate loss
        loss.backward() #  backpropagate the loss
        for f in net.parameters():
            f.data.sub_(f.grad.data * learning_rate) # weight = weight - learning_rate * gradient (Update Weights)
        runningLoss += loss.item()
        
    print('At Iteration : %d / %d  ;  Mean-Squared Error : %f'%(epoch + 1,iterations,
                                                                        runningLoss/(60000/BatchSize)))

At Iteration : 1 / 10  ;  Mean-Squared Error : 0.077912
At Iteration : 2 / 10  ;  Mean-Squared Error : 0.068355
At Iteration : 3 / 10  ;  Mean-Squared Error : 0.067962
At Iteration : 4 / 10  ;  Mean-Squared Error : 0.067775
At Iteration : 5 / 10  ;  Mean-Squared Error : 0.067654
At Iteration : 6 / 10  ;  Mean-Squared Error : 0.067562
At Iteration : 7 / 10  ;  Mean-Squared Error : 0.067487
At Iteration : 8 / 10  ;  Mean-Squared Error : 0.067420
At Iteration : 9 / 10  ;  Mean-Squared Error : 0.067357
At Iteration : 10 / 10  ;  Mean-Squared Error : 0.067295


In [8]:

# Removing the decoder module from the autoencoder
new_classifier = nn.Sequential(*list(net.children())[:-1])
net = new_classifier
new_classifier = nn.Sequential(*list(net[0].children())[:-1])
net = new_classifier
# Adding linear layer for 10-class classification problem
net.add_module('classifier', nn.Sequential(nn.Linear(256, 10),nn.LogSoftmax()))
print(net)
if use_gpu:
    net = net.double().cuda()
else:
    net = net.double()

Sequential(
  (0): Linear(in_features=784, out_features=400, bias=True)
  (1): Tanh()
  (2): Sequential(
    (0): Linear(in_features=400, out_features=256, bias=True)
    (1): Tanh()
  )
  (classifier): Sequential(
    (0): Linear(in_features=256, out_features=10, bias=True)
    (1): LogSoftmax()
  )
)


In [10]:
iterations = 10
learning_rate = 0.1
criterion = nn.NLLLoss()

for epoch in range(iterations):  # loop over the dataset multiple times

    runningLoss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        
        # wrap them in Variable
        if use_gpu:
            inputs, labels = Variable(inputs.view(-1, 28*28).double()).cuda(), Variable(labels).cuda()
        else:
            inputs, labels = Variable(inputs.view(-1, 28*28).double()), Variable(labels)

        net.zero_grad()  # zeroes the gradient buffers of all parameters
        outputs = net(inputs) # forward 
        loss = criterion(outputs, labels) # calculate loss
        loss.backward() #  backpropagate the loss
        for f in net.parameters():
            f.data.sub_(f.grad.data * learning_rate) # weight = weight - learning_rate * gradient (Update Weights)
        runningLoss += loss.item()
        correct = 0
        total = 0
    for data in testloader:
        inputs, labels = data
        if use_gpu:
            inputs, labels = Variable(inputs.view(-1, 28*28).double()).cuda(), labels.cuda()
        else:
            inputs, labels = Variable(inputs.view(-1, 28*28).double()), labels
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    print('At Iteration : %d / %d  ;  Train Error : %f ;Test Accuracy : %f'%(epoch + 1,iterations,
                                                                        runningLoss/(60000/BatchSize),100 * correct /float(total)))
print('Finished Training')

At Iteration : 1 / 10  ;  Train Error : 2.026576 ;Test Accuracy : 51.000000
At Iteration : 2 / 10  ;  Train Error : 1.069804 ;Test Accuracy : 80.000000
At Iteration : 3 / 10  ;  Train Error : 0.643749 ;Test Accuracy : 85.000000
At Iteration : 4 / 10  ;  Train Error : 0.484072 ;Test Accuracy : 88.000000
At Iteration : 5 / 10  ;  Train Error : 0.410531 ;Test Accuracy : 89.000000
At Iteration : 6 / 10  ;  Train Error : 0.371820 ;Test Accuracy : 90.000000
At Iteration : 7 / 10  ;  Train Error : 0.344581 ;Test Accuracy : 90.000000
At Iteration : 8 / 10  ;  Train Error : 0.326761 ;Test Accuracy : 91.000000
At Iteration : 9 / 10  ;  Train Error : 0.312177 ;Test Accuracy : 91.000000
At Iteration : 10 / 10  ;  Train Error : 0.299513 ;Test Accuracy : 91.000000
Finished Training


In [11]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
for data in testloader:
    images, labels = data
    if use_gpu:
        outputs = net(Variable(images.view(-1, 28*28).double().cuda()))
        _, predicted = torch.max(outputs.data.cpu(), 1)
    else:
        outputs = net(Variable(images.view(-1, 28*28).double()))
        _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(BatchSize):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %f %%' % (
        classes[i], 100 * class_correct[i] / float(class_total[i])))

Accuracy of  zero : 0.000000 %
Accuracy of   one : 0.000000 %
Accuracy of   two : 6.000000 %
Accuracy of three : 0.000000 %
Accuracy of  four : 0.000000 %
Accuracy of  five : 0.000000 %
Accuracy of   six : 0.000000 %
Accuracy of seven : 26.000000 %
Accuracy of eight : 1.000000 %
Accuracy of  nine : 0.000000 %
